In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import math

from picking.utils import *

Se obtiene un archivo .txt con:
- Tiempo real del sismo
- Magnitud
- Tiempo en que se debería observar el sismo en cada estación

Todos estos separados en columnas

In [4]:
# Station coordinates
stations_coord = {
    'AC04': (-28.20, -71.07),
    'AC05': (-28.84, -70.27),
    'CO05': (-29.92, -71.24),
    'CO10': (-29.24, -71.46)
}

# P-wave propagation speed
v_P = 8.064

# Read Excel file
df = pd.read_excel('Eventos_24hrs.xlsx')

# Calculate detection times and format DataFrame
formatted_df = calculate_detection_times(df, stations_coord, v_P, magnitude_thr=4)

# Save DataFrame to a CSV file
formatted_df.to_csv('times_events_24hrs_sup40.txt', sep=',', index=False)

Se obtiene un archito txt de catalogs_events_new.xlsx

In [24]:
# Lee el archivo Excel
df = pd.read_excel('catalog_events_new.xlsx', sheet_name="Catalogo_csn")

# Selecciona las columnas que deseas eliminar
columnas = ['n', 'Latitud', 'Longitud', 'Profundidad', 'Esta', 'Primera_obs', 'segunda_Obs', 'tercera_obs']
columnas += [col for col in df.columns if 'Fin' in col]

# Elimina las columnas seleccionadas
df = df.drop(columns=columnas)

# Elimina las filas donde la magnitud sea inferior a 4
df = df[df['Magnitud'] >= 4]

# Ajusta las fechas de las estaciones para que sean iguales en formato a la de la primera columna
columnas_inicio = [col for col in df.columns if 'Inicio' in col]
for col in columnas_inicio:
    df[col] = pd.to_datetime(df[col], format='%H:%M:%S:%f').dt.time
    df[col] = df.apply(lambda row: datetime.combine(row['Fecha UTC ORIGEN'], row[col]), axis=1)
    df[col] = df[col].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S.%fZ'))

# Formatea la columna 'Fecha UTC ORIGEN'
df['Fecha UTC ORIGEN'] = df['Fecha UTC ORIGEN'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S.%fZ'))

# Guarda el DataFrame en un archivo de texto
df.to_csv('catalog_new_events.txt', sep='\t', index=False)


In [42]:
def read_times_new(filename, time_format, station_identifier, skip_header=True):
    with open(filename, 'r') as f:
        # Use '\t' as the delimiter since the header contains tabs
        reader = csv.reader(f, delimiter='\t')

        if skip_header:
            headers = next(reader)

            # Split the headers using tabs and strip whitespace
            headers = [header.strip() for header in headers[0].split('\t')]
        else:
            headers = next(reader)

        print(headers)
        station_index = headers.index(f'Inicio_{station_identifier}')
        print(station_index)

        times = [datetime.strptime(row[station_index], time_format) for row in reader]

        return times

path_new_catalog = "catalog_new_events.txt"
station_identifier = "CO10"
time_format = "%Y-%m-%dT%H:%M:%S.%fZ"

read_times_new(path_new_catalog, time_format = time_format, station_identifier=station_identifier)

['Fecha UTC ORIGEN']


ValueError: 'Inicio_CO10' is not in list

# Esto es solo para el archivo que dura una hora, nada más


In [32]:
# Read Excel file
df = pd.read_excel('Eventos_24hrs.xlsx')
# Tomar solo los eventos entre las horas 00:30 y 01:30
df = df[(df['Fecha UTC'] >= datetime(2021, 7, 4, 1, 00)) & (df['Fecha UTC'] <= datetime(2021, 7, 4, 4, 30))]

# Calculate detection times and format DataFrame
formatted_df_chiquito = calculate_detection_times(df, stations_coord, v_P, magnitude_thr=3.5)

# Save DataFrame to a CSV file
formatted_df_chiquito.to_csv('times_events_24hrs_chiquito.txt', sep=',', index=False)

In [33]:
formatted_df_chiquito

,Fecha UTC,Magnitud,Hora detección estación CO10,Hora detección estación AC04,Hora detección estación AC05,Hora detección estación CO05
0,2021-07-04T01:01:37,5.9,2021-07-04T01:01:43,2021-07-04T01:01:47,2021-07-04T01:01:52,2021-07-04T01:01:52
1,2021-07-04T01:07:22,3.6,2021-07-04T01:07:29,2021-07-04T01:07:31,2021-07-04T01:07:38,2021-07-04T01:07:38
7,2021-07-04T01:29:15,5.8,2021-07-04T01:29:21,2021-07-04T01:29:25,2021-07-04T01:29:30,2021-07-04T01:29:31
8,2021-07-04T01:34:42,4.0,2021-07-04T01:34:48,2021-07-04T01:34:51,2021-07-04T01:34:56,2021-07-04T01:34:58
9,2021-07-04T01:36:16,4.1,2021-07-04T01:36:22,2021-07-04T01:36:24,2021-07-04T01:36:30,2021-07-04T01:36:32
14,2021-07-04T01:50:43,3.6,2021-07-04T01:50:50,2021-07-04T01:50:51,2021-07-04T01:50:57,2021-07-04T01:50:59
16,2021-07-04T01:57:43,3.7,2021-07-04T01:57:48,2021-07-04T01:57:53,2021-07-04T01:57:58,2021-07-04T01:57:58
17,2021-07-04T02:05:56,3.7,2021-07-04T02:06:02,2021-07-04T02:06:05,2021-07-04T02:06:10,2021-07-04T02:06:12
18,2021-07-04T02:07:44,3.5,2021-07-04T02:07:50,2021-07-04T02:07:53,2021-07-04T02:07:59,2021-07-04T02:08:00
19,2021-07-04T02:08:29,3.6,2021-07-04T02:08:34,2021-07-04T02:08:39,2021-07-04T02:08:44,2021-07-04T02:08:44
